In [26]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import json
import os
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
import torch.nn.functional as F
import random

# 0. Device & model
device    = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model     = models.densenet121(weights='IMAGENET1K_V1').to(device)
model.eval()

# Define preprocessing transforms (add resize for ResNet-34)
mean_norms = np.array([0.485, 0.456, 0.406])
std_norms = np.array([0.229, 0.224, 0.225])
plain_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # Ensure image size matches ResNet-34 input
    transforms.ToTensor(),
    transforms.Normalize(mean=mean_norms, std=std_norms)
])

# Load test dataset
dataset_path = "./TestDataSet"  # Update with actual path
dataset = ImageFolder(root=dataset_path, transform=plain_transforms)
dataloader = DataLoader(dataset, batch_size=32, shuffle=False, num_workers=4)

# Load label mappings from JSON file
with open("./TestDataSet/labels_list.json", "r") as f:  # Update with actual JSON path
    label_mappings = json.load(f)

# Create a mapping from ImageFolder indices to ImageNet indices
# ImageFolder assigns indices based on folder names (alphabetical order)
folder_to_idx = {class_name: idx for idx, class_name in enumerate(dataset.classes)}

# Parse the JSON entries and sort by ImageNet index
label_mappings_parsed = []
for entry in label_mappings:
    imagenet_idx, class_name = entry.split(": ")
    label_mappings_parsed.append((int(imagenet_idx), class_name))
label_mappings_parsed.sort(key=lambda x: x[0])  # Sort by ImageNet index

# Sort folder names (WNIDs) to match ImageFolder's ordering
sorted_folders = sorted(dataset.classes)

# Map ImageFolder indices to ImageNet indices
# Assume the sorted WNIDs correspond to the sorted ImageNet indices
idx_to_imagenet_idx = {}
for folder_idx, (imagenet_idx, class_name) in enumerate(label_mappings_parsed):
    if folder_idx < len(sorted_folders):
        idx_to_imagenet_idx[folder_idx] = imagenet_idx
    else:
        print(f"Warning: More ImageNet indices than folders. Extra entry: {imagenet_idx}: {class_name}")

# Debugging: Print the mapping
#print("Mapping (ImageFolder idx -> ImageNet idx):", idx_to_imagenet_idx)

# Check for unmapped ImageFolder indices
all_folder_indices = set(range(len(dataset.classes)))
mapped_indices = set(idx_to_imagenet_idx.keys())
unmapped_indices = all_folder_indices - mapped_indices
if unmapped_indices:
    print(f"Unmapped ImageFolder indices: {unmapped_indices}")
    for idx in unmapped_indices:
        print(f"Index {idx} corresponds to folder: {dataset.classes[idx]}")

# Function to compute top-k accuracy
def compute_topk_accuracy(outputs, labels, k=1):
    batch_size = labels.size(0)
    _, pred = outputs.topk(k, dim=1, largest=True, sorted=True)
    pred = pred.t()
    correct = pred.eq(labels.view(1, -1).expand_as(pred))
    correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
    return correct_k.mul_(100.0 / batch_size).item()



# Evaluate model
top1_correct = 0
top5_correct = 0
total = 0

with torch.no_grad():
    for images, labels in dataloader:
        images = images.to(device)
        # Map ImageFolder labels to ImageNet indices
        imagenet_labels = torch.tensor([idx_to_imagenet_idx[label.item()] for label in labels]).to(device)
        
        outputs = model(images)
        
        # Compute top-1 and top-5 accuracy
        top1_correct += compute_topk_accuracy(outputs, imagenet_labels, k=1)
        top5_correct += compute_topk_accuracy(outputs, imagenet_labels, k=5)
        total += 1

# Calculate average accuracies
top1_t1_accuracy = top1_correct / total
top5_t1_accuracy = top5_correct / total


results = {}
results["Clean"] = (top1_t1_accuracy, top5_t1_accuracy)

# ###################################################################
# #FGSM
# FGSM = ImageFolder(root="./AdversarialTestSet1", transform=plain_transforms)
# F = DataLoader(FGSM, batch_size=32, shuffle=False, num_workers=4)

# # Evaluate model
# top1_correct = 0
# top5_correct = 0
# total = 0

# with torch.no_grad():
#     for images, labels in F:
#         images = images.to(device)
#         # Map ImageFolder labels to ImageNet indices
#         imagenet_labels = torch.tensor([idx_to_imagenet_idx[label.item()] for label in labels]).to(device)
        
#         outputs = model(images)
        
#         # Compute top-1 and top-5 accuracy
#         top1_correct += compute_topk_accuracy(outputs, imagenet_labels, k=1)
#         top5_correct += compute_topk_accuracy(outputs, imagenet_labels, k=5)
#         total += 1

# # Calculate average accuracies
# top1_t1_accuracy = top1_correct / total
# top5_t1_accuracy = top5_correct / total

# results["FGSM(Set 1)"] = (top1_t1_accuracy, top5_t1_accuracy)

# #PGD
# PGD = ImageFolder(root="./AdversarialTestSet2", transform=plain_transforms)
# P = DataLoader(PGD, batch_size=32, shuffle=False, num_workers=4)

# # Evaluate model
# top1_correct = 0
# top5_correct = 0
# total = 0

# with torch.no_grad():
#     for images, labels in P:
#         images = images.to(device)
#         # Map ImageFolder labels to ImageNet indices
#         imagenet_labels = torch.tensor([idx_to_imagenet_idx[label.item()] for label in labels]).to(device)
        
#         outputs = model(images)
        
#         # Compute top-1 and top-5 accuracy
#         top1_correct += compute_topk_accuracy(outputs, imagenet_labels, k=1)
#         top5_correct += compute_topk_accuracy(outputs, imagenet_labels, k=5)
#         total += 1

# # Calculate average accuracies
# top1_t1_accuracy = top1_correct / total
# top5_t1_accuracy = top5_correct / total

# results["PGD(Set 2)"] = (top1_t1_accuracy, top5_t1_accuracy)

# #Patch
# Patch = ImageFolder(root="./AdversarialTestSet4_loss_alpha", transform=plain_transforms)
# P = DataLoader(Patch, batch_size=32, shuffle=False, num_workers=4)

# # Evaluate model
# top1_correct = 0
# top5_correct = 0
# total = 0

# with torch.no_grad():
#     for images, labels in P:
#         images = images.to(device)
#         # Map ImageFolder labels to ImageNet indices
#         imagenet_labels = torch.tensor([idx_to_imagenet_idx[label.item()] for label in labels]).to(device)
        
#         outputs = model(images)
        
#         # Compute top-1 and top-5 accuracy
#         top1_correct += compute_topk_accuracy(outputs, imagenet_labels, k=1)
#         top5_correct += compute_topk_accuracy(outputs, imagenet_labels, k=5)
#         total += 1

# # Calculate average accuracies
# top1_t1_accuracy = top1_correct / total
# top5_t1_accuracy = top5_correct / total

# results["Patch(Set 3)"] = (top1_t1_accuracy, top5_t1_accuracy)


print(f"{'Dataset':<15}{'Top-1':>8}{'Top-5':>8}{'Δ1':>8}{'Δ5':>8}")
print("-"*47)
for name, (t1, t5) in results.items():
    print(f"{name:<15}{t1:8.2f}{t5:8.2f}")


Dataset           Top-1   Top-5      Δ1      Δ5
-----------------------------------------------
Clean             74.26   93.52
